In [1]:
import json
import pandas as pd

In [2]:
def read_json_from_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        json_data = json.load(file)
    return json_data

In [3]:
def json_to_dataframe(json_data):
    df = pd.DataFrame(json_data)
    return df

In [5]:
file_path = '../../data.json'
json_data = read_json_from_file(file_path)
df = json_to_dataframe(json_data)
print(df.info())
print(df.head(5))

                                      title  \
0     BMW Seria 3 320d xDrive M Sport sport   
1                  BMW X3 xDrive20d M Sport   
2          BMW Seria 3 320d Touring M Sport   
3                   BMW Seria 7 740d xDrive   
4                BMW Seria 3 318d Advantage   
...                                     ...   
6395                       BMW X3 xDrive30i   
6396                      BMW X3 28i xDrive   
6397                            BMW X5 4.4i   
6398                            BMW Seria 5   
6399               BMW X3 xDrive20d M Sport   

                                                    url               id  \
0     https://www.otomoto.pl/osobowe/oferta/bmw-seri...  619888553496191   
1     https://www.otomoto.pl/osobowe/oferta/bmw-x3-x...  878738710972359   
2     https://www.otomoto.pl/osobowe/oferta/bmw-seri...  325534139055352   
3     https://www.otomoto.pl/osobowe/oferta/bmw-seri...  586230984198372   
4     https://www.otomoto.pl/osobowe/oferta/bmw-seri...